In [1]:
import sys
sys.path.append('../')

import os
import gc
import torch
import psutil
import pickle
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.nn.functional as F
from sklearn import metrics
from collections import Counter
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from torchvision import models, set_image_backend

import data_utils
import train_utils
import model_utils

%reload_ext autoreload
%autoreload 2

set_image_backend('accimage')

## Load Data

In [2]:
pickle_file = '/n/data_labeled_histopathology_images/COAD/train.pkl'
with open(pickle_file, 'rb') as f: 
    train_embeddings, train_labels, train_jpgs_to_slide = pickle.load(f)
    
pickle_file = '/n/data_labeled_histopathology_images/COAD/val.pkl'
with open(pickle_file, 'rb') as f: 
    val_embeddings, val_labels, val_jpgs_to_slide = pickle.load(f)

## Global Vars

In [3]:
input_size = 2048
hidden_size = 2048
output_size = 1
criterion = nn.BCELoss(reduction='none')

## Baseline

In [4]:
state_dict_file = '/n/tcga_models/resnet18_WGD_10x.pt'
resnet = models.resnet18(pretrained=False)
resnet.fc = nn.Linear(input_size, output_size)
saved_state = torch.load(state_dict_file, map_location=lambda storage, loc:storage)
resnet.load_state_dict(saved_state)
for p in resnet.parameters():
    p.requires_grad = False

In [5]:
resnet_fc = resnet.fc
n_samples = val_jpgs_to_slide.max() + 1
probs_vec = torch.zeros((n_samples, 1)).cuda()
labels_vec = torch.zeros_like(probs_vec).cuda()
val_embeddings = val_embeddings.cuda()
resnet_fc.cuda()

Linear(in_features=2048, out_features=1, bias=True)

In [6]:
resnet_fc.eval()
for idx in range(n_samples):
    with torch.no_grad():
        slide = val_embeddings[val_jpgs_to_slide==idx]
        labels_vec[idx] = val_labels[val_jpgs_to_slide==idx].unique().float().cuda()
        probs_vec[idx] = torch.mean((torch.sigmoid(resnet_fc(slide)) > 0.5).float()) # mean pooling the tile-level predicted labels

In [7]:
loss = criterion(probs_vec, labels_vec) # input, target
fpr, tpr, thresholds = metrics.roc_curve(labels_vec.detach().cpu().numpy(), probs_vec.detach().cpu().numpy()) # y_true, y_score
roc_auc = metrics.auc(fpr, tpr)
print(torch.mean(loss), torch.median(loss), roc_auc)

tensor(0.6194, device='cuda:0') tensor(0.4929, device='cuda:0') 0.7240338164251208


## Attention

In [8]:
state_dict_file = '/n/tcga_models/COAD_attention_model_reworked_5_8_acc.pt'
net = model_utils.Attention(input_size, hidden_size, output_size)
saved_state = torch.load(state_dict_file, map_location=lambda storage, loc:storage)
net.load_state_dict(saved_state)
for p in net.parameters():
    p.requires_grad = False
net.cuda()

Attention(
  (V): Linear(in_features=2048, out_features=2048, bias=True)
  (w): Linear(in_features=2048, out_features=1, bias=True)
  (sigm): Sigmoid()
  (tanh): Tanh()
  (sm): Softmax()
  (linear_layer): Linear(in_features=2048, out_features=1, bias=True)
)

In [9]:
net.eval()
with torch.no_grad():
    for idx in range(n_samples):
        slide = val_embeddings[val_jpgs_to_slide==idx]
        labels_vec[idx] = val_labels[val_jpgs_to_slide==idx].unique().float().cuda()
        probs_vec[idx] = torch.mean((torch.sigmoid(net(slide)) > 0.5).float()) # mean pooling the tile-level predicted labels

In [10]:
loss = criterion(probs_vec, labels_vec) # input, target
fpr, tpr, thresholds = metrics.roc_curve(labels_vec.detach().cpu().numpy(), probs_vec.detach().cpu().numpy()) # y_true, y_score
roc_auc = metrics.auc(fpr, tpr)
print(torch.mean(loss), torch.median(loss), roc_auc)

tensor(7.0762, device='cuda:0') tensor(-0., device='cuda:0') 0.7445652173913043


## Rationales

In [11]:
def pool_fn(x):
    #v,a = torch.max(x,0)
    v = torch.mean(x,0)
    return v

In [12]:
state_dict_file = '/n/tcga_models/COAD_rationale_model_gen_5_9_acc.pt'
output_size = 2
gen = model_utils.Generator(input_size, hidden_size, output_size, dropout=0.5)
saved_state = torch.load(state_dict_file, map_location=lambda storage, loc:storage)
gen.load_state_dict(saved_state)
for p in gen.parameters():
    p.requires_grad = False
gen.cuda()

Generator(
  (d): Dropout(p=0.5)
  (m): ReLU()
  (linear1): Linear(in_features=2048, out_features=2048, bias=True)
  (linear2): Linear(in_features=2048, out_features=2, bias=True)
)

In [13]:
state_dict_file = '/n/tcga_models/COAD_rationale_model_enc_5_9_acc.pt'
output_size = 1
enc = model_utils.Encoder(input_size, hidden_size, output_size, pool_fn, dropout=0.5)
saved_state = torch.load(state_dict_file, map_location=lambda storage, loc:storage)
enc.load_state_dict(saved_state)
for p in enc.parameters():
    p.requires_grad = False
enc.cuda()

Encoder(
  (d): Dropout(p=0.5)
  (m): ReLU()
  (linear1): Linear(in_features=2048, out_features=2048, bias=True)
  (linear2): Linear(in_features=2048, out_features=1, bias=True)
)

In [14]:
gen.eval()
enc.eval()
with torch.no_grad():
    for idx in range(n_samples):
        slide = val_embeddings[val_jpgs_to_slide==idx]
        labels_vec[idx] = val_labels[val_jpgs_to_slide==idx].unique().float().cuda()

        preds = gen(slide)
        sample = torch.argmax(preds, dim=1).float()
        rationale = slide * sample.unsqueeze(1)

        logits = enc(rationale)
        probs_vec[idx] = torch.mean((torch.sigmoid(logits) > 0.5).float()) # mean pooling the tile-level predicted labels

In [15]:
loss = criterion(probs_vec, labels_vec) # input, target
fpr, tpr, thresholds = metrics.roc_curve(labels_vec.detach().cpu().numpy(), probs_vec.detach().cpu().numpy()) # y_true, y_score
roc_auc = metrics.auc(fpr, tpr)
print(torch.mean(loss), torch.median(loss), roc_auc)

tensor(7.7502, device='cuda:0') tensor(-0., device='cuda:0') 0.7288647342995169
